# Course 1: Lidar
## Part 3: Clustering Obstacles
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Sensor Fusion Nanodegree programme offered at Udacity.

## Objectives

## 1. Introduction

### 1.1: Euclidean Clustering with K-D Tree

#### Euclidean clustering algorithm

Euclidean clustering is a type of [cluster analysis](https://en.wikipedia.org/wiki/Cluster_analysis) algorithm used to detect and group nearby points in a dataset. Using a [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) heuristic, each point's membership to its neighbours is considered. Points which are "close" in distance to each otoher are grouped together to form a local cluster. Through the use of a [nearest neighbour search](https://en.wikipedia.org/wiki/Nearest_neighbor_search), this "closeness" heuristic is explored, and the resulting set of clusters is approximated in the fashion of an optimisation problem. In other words, the "most optimal" set of resulting clusters is determined by grouping together the points whose distance between each other is minimal.

In our application, we use Euclidean clustering to group the LiDAR points belonging to the distinct objects in our scene — the surrounding vehicles. We make use of the Point Cloud Library (PCL), which provides us with the [`pcl::EuclideanClusterExtraction`](https://pointclouds.org/documentation/tutorials/cluster_extraction.html) class to perform the clustering. Using a [K-D Tree](https://en.wikipedia.org/wiki/K-d_tree) allows us to "speed up" the nearest neighbour search by organising points into a hierarchial strucutre, therefore restricting the number of total points that must be "explored" when forming each cluster. The K-D Tree is a special case of the [binary space partitioning](https://en.wikipedia.org/wiki/Binary_space_partitioning) tree structure, which "splits" the point space into branches based on the median values of the points along the chosen dimension(s). Once the tree is formed using the entire set of possible points, "searching" for nearby points becomes a matter of tree traversal, where the distance is checked against the nodes of the tree in a recursive manner. With the K-D Tree structure, we are able to more-efficiently perform "look-up" in $O\left(\mathrm{log}n\right)$ time complexity.

### 1.2: Using the K-D Tree

#### Insertion

The first point inserted into the K-D Tree naturally becomes the "root" node; at the very beginning when the tree is empty, the root node is `NULL`. After the first point is inserted, the root node is populated. In our case, the tree splits the $x$-axis region with the $x$-coordinate value of the first point inserted. In other words, for a first point $n_{1} = \left(-6.2, 7.0\right)$, we have a K-D Tree (of `depth = 0`) whose root is equal to the inserted point. This K-D Tree will have _left_ nodes with future points containing $x$-axis values _less than_ $-6.2$, and _right_ nodes with future points containing $x$-axis values _greater than_ $-6.2$. For the next point inserted (at `depth = 1`), we will split along the $y$-axis. For example, the point $n_{2} = \left(-6.3, 8.4\right)$ becomes the _left_ child node of the root, since its $x$-axis value $-6.3$ is less than the root with value $-6.2$. Our third point, $n _{3} = \left(-5.2, 7.1\right)$, then becomes the _right_ child node of thre root and remains at a `depth = 1` in the tree. A fourth point $n _{4} = \left(-5.7, 6.3\right)$, is inserted into the three. This time, the splitting axis is along the $y$-axis, which means that this node will be inserted at `depth = 2` in the tree, and become the _left_ child node of the third node $n_{3}$, since this fourth point has a $y$-coordinate value of $6.3$ which is _less than_ $7.1$, the $y$-coordinate value of the third point $n_{3}$.


This gives us the tree structure shown in Figure 1 below:

<img src="figures/2024-05-13-Figure-1-Using-the-K-D-Tree-Insertion.png" alt="Figure 1. Using the K-D Tree — Insertion (shown in the figure is a simple K-D Tree with four nodes; n_{1} appearing at the root with `depth = 0`, n_{2} and n_{3} at `depth = 1` as left- and right child nodes, respectively, and n_{4} at `depth = 2` as the left child node of n_{3})." height="65%" width="65%">

$$\textrm{Figure 1. Using the K-D Tree — Insertion.}

##### Quiz Question #1

**Question #1**: Given a node with values $\left(7.2, 6.1\right)$, where in the above tree (shown in Figure 1) should the node be inserted at?

**Answer #1**: At position $\textrm{D}$.

**Explanation**: The node $\left(7.2, 6.1\right)$ will be inserted at position $\textrm{D}$ in the tree. This is because we traverse the tree from the root by first comparing the node's $x$-coordinate value, $7.2$, with the root, $-6.2$. Since the node with value $7.2$ is _greater than_ the root's $x$-value, we "split" to the right branch. At the next level (`depth = 1`), we now consider the node's $y$-coordinate value of $6.1$. Since this value is _less than_ the right child node's $y$-coordinate value of $7.1$, we move to the left sub-branch. At `depth = 2`, we are left with two possible positions, $\textrm{C}$ and $\textrm{D}$. By comparing the node's $x$-coordinate value of $7.2$ against the already-inserted node's value of $-5.7$, we determine the position at which the given node will be inserted into the tree is at location $\textrm{D}$, since the value $7.2$ is _greater than_ $-5.7$. As you may have observed in this exercise, the "axis" we split at alternates with each iteration of `depth` in the tree — starting with $x$-axis at `depth = 0`, then $y$-axis at `depth = 1`, and so on... 

##### Performance Improvements

By "balancing" the tree we can improve search time. To do so, insert points that _evenly_ split the $x$- and $y$-region values by sorting the points to be inserted into the tree and selecting the _median_ value along the respective axis we are considering (depending on the `depth` we are considering at the current iteration). For instance, consider the following four points:
$$\begin{Bmatrix}
    \left(-6.3, 8.4\right), \left(-6.2, 7.0\right), \left(-5.2, 7.1\right), \left(-5.7, 6.3\right)\end{Bmatrix}.$$
The first point to be inserted into the tree shown in Figure 1 would therefore be $\left(-5.2, 7.1\right)$. This is because this point along the $x$-axis is equal to the median of the set. For the next point to insert, we select the median value along the alternate $y$-axis. This gives us $\left(-6.2, 7.0\right)$, the second point to insert. For the third point to insert, we switch back to the $x$-axis and consider the next median $x$-value between the two remaining points. This gives us $\left(-5.7, 6.3\right)$. Then we conclude the insert operation with the fourth and final point remaining — $\left(-6.3, 8.4\right)$.

By selecting the median value with respect to the current splitting axis we are considering, we can more-optimally split the region at each iteration and effectively improve the search time during later use.

#### Search

Now that we understand how to construct a K-D Tree and populate it with 2D point data, we will now learn to search through the tree. With a [nearest neighbour search](https://en.wikipedia.org/wiki/Nearest_neighbor_search), we discover a set of points that are closest in proximity to our "target" point. By using a K-D Tree structure, we can "speed up" the search time by discarding large "regions" of points that are likely to not "belong" to the set of points in which our target point is closest to. In order to examine the nearby points to our target, we consider a heuristic value known as the distance threshold (`distanceTol`). This threshold represents the maximum distance away from our "target" point we want to consider when associating nearby points. This "region" of similiarity is created such that any points within a `2 * distanceTol` length away from our target point will be considered as a "neighbour" of the target. If the current point being examined _does not_ fall within the `2 * distanceTol` length, then that point and _all successive points_ along its branch can be discarded (i.e., we ignore all points "further" away from the target which lie outside the region defined by `2 * distanceTol`).

#### Euclidean Clustering

We can extend the core functions we have previously constructed for the K-D Tree to now perform Euclidean Clustering of points in a "point cloud". For each point in the point cloud, we use the `insert` and `search` functions to construct "branches" of the K-D Tree and search the branches for neighbouring points. Ultimately, with the `euclideanCluster` function, we will group together the points from the point cloud to form individual clusters.

## 2. Programming Task

### Euclidean Clustering with KD-Tree using Point Cloud Library (PCL)

#### E1.3.1: `Clustering`

By following the [Euclidean Cluster Extraction](https://pointclouds.org/documentation/tutorials/cluster_extraction.html) tutorial from Point Cloud Library (PCL), we will be able to group together individual LiDAR point returns to form "clusters". These clusters can be then used to identify objects in the driving scene, such as other vehicles, pedestrians or VRUs (vulnerable road users, e.g., bicyclists).  

##### The `Clustering` function

```cpp
// From J. Moran's `src/processPointClouds.cpp`
// Credit: https://github.com/jonathanloganmoran/ND313-Sensor-Fusion-Engineer/blob/1.3/1-Lidar/1-1-Lidar-Obstacle-Detection/src/processPointClouds.cpp
```

```cpp
// In `src/processPointClouds.cpp`:


template<typename PointT> std::vector<
    typename pcl::PointCloud<PointT>::Ptr
> ProcessPointClouds<PointT>::Clustering(
    typename pcl::PointCloud<PointT>::Ptr cloud,
    float clusterTolerance,
    int minSize,
    int maxSize
) {
    // Time clustering process
    auto startTime = std::chrono::steady_clock::now();
    std::vector<typename pcl::PointCloud<PointT>::Ptr> clusters;
    /*** E1.3.1: Euclidean clustering with PCL. ***/
    // TODO:: Fill in the function to perform euclidean clustering to group detected obstacles
    // Creating the KD-Tree object for the search method of the extraction
    pcl::search::KdTree<pcl::PointXYZ>::Ptr tree(
        new pcl::search::KdTree<pcl::PointXYZ>
    );
    // Creating the Euclidean clustering class instance
    pcl::EuclideanClusterExtraction<pcl::PointXYZ> ec;
    // Setting the input cloud for the KD-Tree
    // NOTE: We assume the ground plane has been "filtered" out
    tree->setInputCloud(cloud);
    // Configuring the clustering parameters
    std::vector<pcl::PointIndices> clusterIndices;
    ec.setClusterTolerance(clusterTolerance);
    ec.setMinClusterSize(minSize);
    ec.setMaxClusterSize(maxSize);
    ec.setSearchMethod(tree);
    ec.setInputCloud(cloud);
    ec.extract(clusterIndices);
    // Performing the clustering with Euclidean distance
    for (const auto& cluster : clusterIndices) {
        // Creating a new point cloud instance for the current cluster
        pcl::PointCloud<pcl::PointXYZ>::Ptr cloudCluster(
            new pcl::PointCloud<pcl::PointXYZ>
        );
        for (const auto& idx : cluster.indices) {
            // Copying over the indices of the current cluster
            cloudCluster->push_back(
                (*cloud)[idx]
            );
        }
        // Setting the cluster parameters
        cloudCluster->width = cloudCluster->size();
        cloudCluster->height = 1;
        cloudCluster->is_dense = true;
        std::cout << "PointCloud representing the Cluster: "
                  << cloudCluster->size() << " data points.\n";
        // Adding cluster to return vector
        clusters.push_back(cloudCluster);
    }
    auto endTime = std::chrono::steady_clock::now();
    auto elapsedTime = std::chrono::duration_cast<
        std::chrono::milliseconds
    >(endTime - startTime);
    std::cout << "Clustering took " 
              << elapsedTime.count() << " milliseconds,"
              << " and found " << clusters.size() << " clusters.\n";
    return clusters;
}
```

##### Testing the `Clustering` function

To run the above `Clustering` algorithm, call the function from inside `src/environment.cpp` as follows:

```cpp
// From J. Moran's `src/environment.cpp`:
// Credit: https://github.com/jonathanloganmoran/ND313-Sensor-Fusion-Engineer/blob/1.3/1-Lidar/1-1-Lidar-Obstacle-Detection/src/environment.cpp
```

```cpp
// In `src/environment.cpp`:

void simpleHighway(
    pcl::visualization::PCLVisualizer::Ptr& viewer
) {
    // ..
    /** E1.3.1: Euclidean clustering with PCL. **/
    // Defining the clustering parameters
    double distanceTol = 1.0;           // Euclidean threshold (m)
    int minSize = 3;
    int maxSize = 30;
    // Performing the clustering
    std::vector<
        pcl::PointCloud<pcl::PointXYZ>::Ptr
    > cloudClusters = pointProcessorXYZ.Clustering(
        segmentCloud.first,
        distanceTol,
        minSize,
        maxSize
    );
    // ..
}
```

We define our input arguments to the `Clustering` function:
* `distanceTol` — Threshold distance (in metres) for determining point candidates, i.e., whether points should be considered part of the same cluster. Points separated by a greater distance than this threshold will not be grouped together;
* `minSize` — The minimum number of points assumed to belong to an individual cluster. By setting a "minimum" number of points for each "cluster", we can prevent forming clusters which are too sparse, which might be related to noise and not representative of actual objects in the scene;
* `maxSize` — The maximum number of points assumed to belong to an individual cluster. By setting a "maximum" number of points for each "cluster", we can prevent forming clusters which are too dense, which might erroneously combine points from multiple distinct objects.

By carefully selecting these parameters, we can attempt to avoid clustering points which might have been generated from noise (either from sensor inaccuracies) or are belonging to non-objects or multiple overlapping objects.

#### E1.3.2: Visualising the clusters

In this part we simply render the clusters we generated above using the `Clustering` function. To display these point clusters, we will use the PCL Viewer canvas. To make the clusters "stand out", we will assign each a unique RGB-valued colour, then render the points onto the canvas.

```cpp
// From J. Moran's `src/environment.cpp`:
// Credit: https://github.com/jonathanloganmoran/ND313-Sensor-Fusion-Engineer/blob/1.3/1-Lidar/1-1-Lidar-Obstacle-Detection/src/environment.cpp
```

```cpp
// In `src/environment.cpp`:

void simpleHighway(
    pcl::visualization::PCLVisualizer::Ptr& viewer
) {
    // ..
    /** E1.3.2: Visualising the clusters found. **/
    // Using a unique identifer for each cluster found
    int clusterId = 0;
    // Set of colours to use when rendering the clusters found
    std::vector<Color> colors = {
        Color(1, 0, 0),
        Color(0, 1, 0),
        Color(0, 0, 1)
    };
    // Rendering each cluster onto the PCL viewer
    for (pcl::PointCloud<pcl::PointXYZ>::Ptr cluster : cloudClusters) {
        std::cout << "cluster size: ";
        pointProcessorXYZ.numPoints(cluster);
        renderPointCloud(
            viewer,
            cluster,
            "obstCloud" + std::to_string(clusterId),
            colors[clusterId]
        );
        ++clusterId;
    }
    // ..
}
```

<img src="figures/2024-05-13-Figure-1-Euclidean-Clustering-Results-Visualised.png" alt="Figure 1. Euclidean Clustering Results — Visualised (shown are two 'blobs' of points, one in the colour green with three distinct points; the other in the colour red with four distinct points)." height="70%" width="70%">

$$\textrm{Figure 1. Euclidean Clustering Results — Visualised.}$$

Shown in the above figure are two 'blobs' of points (the unique "clusters" found). One cluster, shown in the colour _green_, contains a set of three distinct points. The other cluster, shown in the colour _red_, contains a set of four distinct points.

#### E1.3.3: `insert`

In this exercise we define the `insert` function, which will be used to insert new nodes into our tree structure. This function will recursively traverse the tree until a `NULL`-valued "node" has been found. In this case, where the discovered "node" is `NULL`, we update the "empty" node's address to be equal to the "memory address" of a new `Node` instance. With the new `Node` instantiated, we set its value (i.e., the `point` member variable) to be equal to the 2D Cartesian `point` coordinate we wish to insert into our K-D Tree at the `NULL` location found. We also assign the new `Node` an `id` value which corresponds to the current "node" we are inserting into the tree, i.e., a sequential counter determining the current node we are working with.

To achieve the recursive nature of the intended `insert` function, we must create a "helper" function that performs the actual recurisve tree traversal operation. That is, we create an [overloaded function](https://www.geeksforgeeks.org/function-overloading-c/) which contains two additional input arguments: `node` and `depth`. These two arguments allow us to specify the current node and the "depth" of the tree we are considering at the respective iteration of the traversal. Together, these additional arguments allow us to "update" the memory address of the tree "root" to point to the current `node`, plus use the `depth` argument to determine which branch of the sub-tree to "split" next, i.e., whether we should consider the current node's left- or right child. The `depth` value determines which axis, $x$- or $y$-, we will "look at" and compare the values to. For further explanation of this mechanism, see the _Insertion_ sub-section of _Part 1.2: Using the K-D Tree_ above.      

##### The `insert` algorithm

```cpp
// In J. Moran's `src/quiz/cluster/kdtree.h`
// Credit: 
```

```cpp
// In `src/quiz/cluster/kdtree.h`:

struct KdTree {
	// ..
	void insert(
		Node *&node,
		int depth,
		std::vector<float> point, 
		int id
	) {
		/** E1.3.3: Inserting a new `Node` into the tree. **/
		/* Traversing the tree until an "empty" node is found */
		// Determining which of the two coordinate axes to "split" on
		// i.e., we consider either the $x$- or $y$-axis value at this iteration
		uint axis = depth % 2;
		// Using the point value to determine where the node should be inserted
		if (node == NULL) {
			// CASE 1: Found an "empty" node,
			// Insert new `Node` into this location.~
			node = new Node(
				point,
				id
			);
		}
		// CASE 2: Determine which child node to branch to
		// Depending on the point value given, choose left or right
		else if (node->point[axis] > point[axis]) {
			// Value of point to insert is less than current node,
			// i.e., branch to left child node.
			insert(
				node->left,
				depth + 1, 
				point, 
				id
			);
		}
		else if (node->point[axis] < point[axis]) {
			// Value of point to insert is greater than current node,
			// i.e., branch to right child node.
			insert(
				node->right,
				depth + 1,
				point, 
				id
			);
		}
		else {
			// ERROR; should not occur.
			// CANDO: Handle error with case(s).
		}
	}
	void insert(
		std::vector<float> point,
		int id
	) {
		// Begin tree traversal at the initial depth
		int depth = 0;
		// Recursive function call to traverse the tree and insert a new node
		insert(
			this->root,
			depth,
			point,
			id
		);

	}
	// ..
};
}
```

##### Testing the `insert` algorithm

To test the `insert` algorithm above, run the following console commands from inside the [`src/quiz/cluster`]() sub-directory:

```cpp
// In J. Moran's `src/quiz/cluster/`
// Credit: 
```

```console
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/#  mkdir build && cd build
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/build#  cmake ..
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/build#  make
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/build#  ./quizCluster
```

These commands will compile the code from the [`kdtree.h`]() file and provide you with a test case containing a number of points to be inserted into the tree. In this `cmake` script is also a function which is called to visualise the "output" — the 2D points plotted onto a coordinate frame, together with the lines of intersection used to represent the axes in which the K-D Tree is "split" at each iteration. This graphical output is replicated in Figure 2 shown below:  

<img src="figures/2024-05-13-Figure-2-K-D-Tree-Insertion-Visualised.png" alt="Figure 2. K-D Tree: Insertion — Visualised (2D points are shown on the coordinate plane as 'white' dots, the 'splitting' criteria, i.e., which axis the tree 'branched' on with each iteration, is shown as either the 'red'- or 'blue' coloured lines)." width="70%" height="70%">

$$\textrm{Figure 2. K-D Tree: Insertion — Visualised.}$$

Shown in Figure 2 is the visualisation of our K-D Tree populated with "test data", i.e., a set of 2D points, each intersected by a coloured line indicating the axis, either $x$- or $y$, used as the "split" criteria when the point was inserted into the K-D Tree. Each 2D point from our test data (shown in Figure 2 in the colour _white_) is rendered onto a Cartesian coordinate plane. The line intersecting the point, shown in either _red_ or _blue_, indicates which axis, either $x$- or $y$-, was considered at the time the point was inserted into the tree. This axis alternates with each iteration of `depth`. For the very first node `id = 0` to insert into the tree (i.e., the "root"), the `depth` of the KD-Tree will be `0`, and thus the `axis` we consider to make the "split" is calculated as `axis = 0 % 2`, which results in `axis = 0`. Therefore, we "look" at the $x$-coordinate value of our point to insert, then compare it to the existing "node" in the tree, to determine whether to branch left or right. In the case of the root node, we have no "root" node to compare to, so instead we insert the new node directly into the empty K-D Tree at the root. For the next iteration with node `id = 1`, we consider the K-D Tree at a new `depth = 1`. To determine where to insert the new node, we must determine whether we "branch" to the left- or to the right from the root node. Therefore, we look at the `axis = 1 % 2`, which gives us a value `axis = 1`, meaning that we consider the $y$-coordinate value of the "root" node, and compare it against our $y$-value of the `point` to insert. Assuming our node to insert has a $y$-value _greater than_ the root, we branch right. Otherwise, we branch left. This splitting criteria switches with each `depth` in the tree, and is used to determine where we insert each new node into the K-D Tree.   

#### E1.3.4: `search`

##### The `search` algorithm 

```cpp
// From J. Moran's `src/quiz/cluster/kdtree.h`
// Credit: 
```

```cpp
// In `src/quiz/cluster/kdtree.h`:
#include <cmath>

struct KdTree {
	// ..
	bool withinBoundingBox(
		std::vector<float> &target,
		Node *&node,
		float distanceTol
	) {
		// Determining if point is not within "bounding box" of target point,
		// "Bounding box" region spans all coordinate axes of the points
		for (int i = 0; i < node->point.size(); i++) {
			if (
				std::abs(node->point[i] - target[i]) > distanceTol
			) {
				return false;
			}
		}
		// Otherwise, point is considered "near" the `target`
		return true;
	}
	float euclideanDistance(
		std::vector<float> &target,
		std::vector<float> &candidate
	) {
		float sum = 0.0;
		// Checking if both points are of equal dimensions
		if (target.size() != candidate.size()) {
			std::cerr << "Euclidean distance cannot be computed,"
					  << "`target` and `candidate` points are not of equal dimensions.\n";
			return sum;
		}
		// Computing the Euclidean distance between the two points
		for (int i = 0; i < target.size(); i++) {
			sum += std::pow(
				target[i] - candidate[i],
				2
			);
		}
		return std::sqrt(sum);
	}
	void search(
		Node *&node,
		int depth,
		std::vector<int> &ids,
		std::vector<float> target,
		float distanceTol
	) {
		/** E1.3.4: Searching the K-D Tree for nearest neighbours. **/
		// Determining which of the two coordinate axes to "split" on
		// i.e., we consider either the $x$- or $y$-axis value at this iteration
		uint axis = depth % 2;
		// Grabbing next node in tree
		if (node == NULL) {
			// Error; reached end of tree
			return;
		}
		// Performing sanity check
		if (withinBoundingBox(target, node, distanceTol)) {
			// Point is "near" `target`,
			// Checking if distance is within threshold
			float dist = euclideanDistance(target, node->point);
			if (dist <= distanceTol) {
				// Found neighbouring point,
				// Inserting node `id` into neighbours list
				ids.push_back(node->id);
			}
		} // Otherwise, skipping distance calculation and branching
		// Checking the boundary conditions
		if (
			(target[axis] - distanceTol) <= node->point[axis]
		) {
			// Branching to the left
			search(
				node->left,
				depth + 1,
				ids,
				target,
				distanceTol
			);
		}
		if (
			(target[axis] + distanceTol) > node->point[axis]
		) {
			// Branching to the right
			search(
				node->right,
				depth + 1,
				ids,
				target,
				distanceTol
			);
		}
	}
	std::vector<int> search(
		std::vector<float> target, 
		float distanceTol
	) {
		std::vector<int> ids;
		int depth = 0;
		// Recursive function call to traverse the tree and return neighbours
		search(
			this->root,
			depth,
			ids,
			target,
			distanceTol
		);
		return ids;
	}
	// ..
};
```

##### Testing the `search` algorithm

To test the `search` algorithm above, run the following console commands from inside the [`src/quiz/cluster`]() sub-directory:

```console
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/#  mkdir build && cd build
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/build#  cmake ..
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/build#  make
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/build#  ./quizCluster
```

Verify that when the code is run, `line 115` of the `cluster.cpp` file produces the following console output:
```cpp
Test Search
0,1,2,3,
```

You may also experiment with different point values in the function call to `search` on `line 115` in `cluster.cpp`. Use target points that are close to points in the K-D Tree. If the distance tolerance (`distanceTol`) is large enough, then those expected nearby point `id`s should be returned.  

Using a `distanceTol` value of `3.0` and a `target` point of `{-6, 7}`, we obtain a search result containing indices `0,1,2,3`. In other words, we are able to find four neighbouring points to our `target` when using a `distanceTol` of `3.0`. This concludes E1.3.4.

#### E1.3.5: `euclideanCluster`

##### The `euclideanCluster` algorithm

```cpp
// From J. Moran's `src/quiz/cluster/cluster.cpp`
// Credit: 
```

```cpp

// In `src/quiz/cluster/cluster.cpp`:

std::vector<std::vector<int>> cluster(
	int idx,
	const std::vector<std::vector<float>> &points,
	std::vector<int> &c,
	std::vector<bool> &visited,
	KdTree *tree,
	float distanceTol
) {
	// Marking current point as "visited"
	visited[idx] = true;
	// Adding the point index to the "cluster" (assignment)
	c.push_back(idx);
	// Performing the K-D Tree search for neighbouring points
	std::vector<int> idxs = tree->search(points[idx], distanceTol);
	// Recursively "building out" the K-D Tree for each neighbouring point
	for (int i = 0; i < idxs.size(); i++) {
		// Fetching the next neighbouring point's index
		int idxPoint = idxs[i];
		if (!visited[idxPoint]) {
			cluster(
				idxPoint,
				points,
				c,
				visited,
				tree,
				distanceTol
			);
		}
	}
}

std::vector<std::vector<int>> euclideanCluster(
	const std::vector<std::vector<float>> &points, 
	KdTree* tree, 
	float distanceTol
) {
	/** E1.3.5: Euclidean Clustering with the K-D Tree **/
	// TODO: Fill out this function to return list of indices for each cluster
	std::vector<std::vector<int>> clusters;
	// Creating list of "processed" indices
	std::vector<bool> visited{points.size(), false};
	// Forming "clusters" for each point in the point cloud
	for (int i = 0; i < points.size(); i++) {
		// Skipping point if already processed
		if (visited[i]) {
			continue;
		}
		// Creating a new `cluster` and finding neighbouring points
		std::vector<int> c;
		cluster(i, points, c, visited, tree, distanceTol);
		// Adding resulting cluster to vector
		clusters.push_back(c);
	}
	return clusters;
}
```

##### Testing the `euclideanCluster` algorithm

To test the `euclideanCluster` algorithm above, run the following console commands from inside the [`src/quiz/cluster`]() sub-directory:

```console
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/#  mkdir build && cd build
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/build#  cmake ..
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/build#  make
root@foobar:/../1-1-Lidar-Obstacle-Detection/src/quiz/cluster/build#  ./quizCluster
```

Verify that when the code is run, `line 182` of the `cluster.cpp` file produces the following console output:
```cpp
Test Search
0,1,2,3,
clustering found 5 and took 0 milliseconds
```

You may also experiment with different point values in the function call to `euclideanCluster` on `line 182` in `cluster.cpp`.

<img src="figures/2024-05-13-Figure-3-Euclidean-Clustering-K-D-Tree-Results-Visualised.png" alt="Figure 3. K-D Tree: Euclidean Clustering — Visualised." width="70%" height="70%">

$$\textrm{Figure 3. K-D Tree: Euclidean Clustering — Visualised.}$$

In the above Figure 3, we see the results of the Euclidean Clustering algorithm on the set of 2D points, overlaid with the splitting axes (in blue and red) of our K-D Tree traversal. 

## 3. Closing Remarks

#### Alternatives
* Experiment with different `distanceTol` values and re-run the `search` algorithm (`line 115` in `cluster.cpp`);


#### Extensions of task
* Modify `euclideanCluster` to support 3D point values;

## 4. Future Work
* ⬜️
* ✅

## Credits

This assignment was prepared by Aaron Brown and Michael Maile of Mercedes-Benz Research & Development of North America (MBRDNA), 2021 (link [here](https://learn.udacity.com/nanodegrees/nd313/)).


References
* [1] .


Helpful resources:
* []().